In [5]:
import os
import time
import schedule
from watchdog.observers import Observer 
from watchdog.events import FileSystemEventHandler, FileCreatedEvent
import pandas as pd
import shutil
import re


In [8]:
path_to_watch='C:/Users/..'
path_to_dest='C:/Users/dest_eg..'

In [7]:
name_patterns = {r'^file_\d{8}.*.xlsx$':'new_name.xlsx'}

In [10]:
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return
        src_file=event.src_path
        dest_folder=path_to_dest
        original_file_name=os.path.basename(src_file)

        for pattern, new_name in name_patterns:
            if re.match(pattern, original_file_name):
                new_file_name=new_name
                shutil.move(src_file, os.path.join(dest_folder, new_file_name))

    def process_existing_files(self, path_to_watch):
        for filename in os.listdir(path_to_watch):
            file_path = os.path.join(path_to_watch, filename)
            if os.path.isfile(file_path):
                event = FileCreatedEvent(file_path)
                self.on_created(event)


        

In [ ]:
def run_observer(path_to_watch):
    event_handler = MyHandler()
    event_handler.process_existing_files(path_to_watch)
    observer = Observer()
    observer.schedule(event_handler, path_to_watch, recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        observer.join()

run_observer(path_to_watch)